# Imports

In [ ]:
!pip install opencv-python
!sudo apt update && sudo apt install python3-dev python3-pip build-essential

import cv2
from google.colab import drive
import pandas as pd
import os
from PIL import Image


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,748 kB]
Get:13 http://security.ubuntu.

# Connect and load dataset from google drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loaded_labels=[ "RockClimbingIndoor", "Lunges", "Fencing"]

In [18]:
dataset_path = '/content/drive/MyDrive/DAT255_Semesterproject/UCF50/'


# Create a list to store the dataset (video paths and labels)
dataset = []

# Iterate through the selected labels
for label in loaded_labels:
    # Construct the path to the label's subfolder
    label_path = os.path.join(dataset_path, label)

    # Check if the label folder exists
    if os.path.exists(label_path):
        # Iterate through the video files in the label folder
        for filename in os.listdir(label_path):
            # Check if the file is a video (you can adjust the extensions as needed)
            if filename.endswith(('.avi', '.mp4')):  # Add other video extensions if needed
                # Construct the full path to the video file
                video_path = os.path.join(label_path, filename)

                # Append the video path and label to the dataset
                dataset.append((video_path, label))

# Now you have a list of tuples, where each tuple contains:
# (video_path, label)

# Example: Print the first 5 entries in the dataset
print(dataset[:5])

[]


# Install Detectron2

In [19]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment


In [20]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.6 ; cuda:  cu124
detectron2: 0.6


In [23]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Test model on a video from the dataset

In [22]:
image_path = dataset[0][0]

video = cv2.VideoCapture(image_path)
success, image = video.read()
if success:
    outputs = predictor(image)
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
else:
    print("Failed to load the video or extract the frame.")
video.release()


IndexError: list index out of range

In [ ]:
import cv2
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Load the configuration for the keypoint detection model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Set the confidence threshold
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

# Create a predictor object
predictor = DefaultPredictor(cfg)

# Get the video path from your dataset
video_path = dataset[0][0]  # Use the first video in the dataset

# Open the video file
video = cv2.VideoCapture(video_path)

# Get video properties for output video
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Create a VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use appropriate codec
output_video_path = 'output_keypoints.mp4'  # Choose an output path
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while(video.isOpened()):
    ret, frame = video.read()
    if ret:
        # Perform keypoint detection on the frame
        outputs = predictor(frame)

        # Visualize the predictions
        v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out_frame = v.draw_instance_predictions(outputs["instances"].to("cpu")).get_image()[:, :, ::-1]

        # Write the frame to the output video
        out.write(out_frame)

        # Display the frame (optional)
        cv2_imshow(out_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release resources
video.release()
out.release()
cv2.destroyAllWindows()
print(f"Keypoint detection video saved to: {output_video_path}")

[03/17 11:54:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl ...


model_final_a6e10b.pkl: 237MB [00:01, 131MB/s]                           


IndexError: list index out of range